In [1]:
import WorkflowManager
import PromptsReader

manager = WorkflowManager.WorkflowManager()

### Example Process 1 
process = PromptsReader.read_text_file(PromptsReader.TextFiles.PROCESS_1.value)

### extract workflow model and activities labeled with actors
workflow_modelJson, activities_mapped = manager.create_workflow_model(process, image_path="process1.png")


Assume the following for the definition of Workflow Models: A Workflow Model consists of the following node types: Task, XOR-Split, XOR-Join, Start-Node, End-Node. Nodes of type Start-Node have exactly one successor node and no predecessor node. Nodes of type End-Node have exactly one predecessor node and no successor node. Nodes of type Task have exactly one successor node and exactly one predecessor node. Tasks can be followed by gateways (XOR-Split or XOR-Join) or by other tasks. Nodes of type XOR-Split have one predecessor node and two successor nodes (successor_true, successor_false) and a boolean condition. Nodes of type XOR-Join have two predecessor nodes (predecessor_true, predecessor_false) and one successor node. If a process contains a XOR-Split gateway, it must also contain a XOR-Join gateway. 
### instruction: 
The MPON sends the dismissal to the MPOO. The MPOO reviews the dismissal. If the dismissal is invalid, the MPOO opposes the dismissal. Otherwise the MPOO confirmes 

2023-10-10 11:01:34 [INFO] : Took [143.41ms] to generate 'process1.png' diagram


In [6]:
import json
workflowJ = json.loads(workflow_modelJson)

print (print(type(workflowJ)))

<class 'dict'>
None


In [7]:
### read in a local text file as dict
import json
import Process_Map_Creator
with open("prompts\\workflow1.txt", "r") as f:
    data = json.load(f)

Process_Map_Creator.generate_process_map(workflowJ, "workflow1.png")

2023-10-10 10:58:16 [INFO] : Took [180.96ms] to generate 'workflow1.png' diagram


In [3]:
print (activities_mapped.to_json())

[{"id": "T1", "label": "places an order", "actor_label": "A Customer", "actor_id": "C1"}, {"id": "T2", "label": "checks the payment", "actor_label": "The Warehouse", "actor_id": "W1"}, {"id": "T3", "label": "ships the order", "actor_label": "The Warehouse", "actor_id": "W1"}, {"id": "T4", "label": "rejects the order", "actor_label": "The Warehouse", "actor_id": "W1"}]
